# Score Scenarios

Run the cells below to score the configured scenarios. Any unscored scenarios will be scored.

To re-score a scenario, open its corresponding file geodatabase and delete its `scores` and `scores_summary` table.

In [3]:
import sys
import os
import arcpy

import pandas as pd
from arcgis.features import SpatialDataFrame

src = os.path.join(os.path.abspath("."), 'src')
if src not in sys.path:
    sys.path.append(src)
    
from ato_tools import ato

if 'ato_tools' in sys.modules:
    import importlib
    importlib.reload(ato)

base_path = os.path.abspath(".")

baseline_gdb = "baseline.gdb"

In [20]:
scenarios = list()
for mode in ['Cycling', 'Driving', 'Transit', 'Land_Use']:
    for file in os.listdir(os.path.join('scenario', mode)):
        d = os.path.join(base_path, os.path.join('scenario', mode), file)
        if os.path.isdir(d) and d.endswith('.gdb'):
            scenarios.append({"name": file[:-4],
                              'gdb': d,
                              'mode': mode})
            
modal_scenarios = [x for x in scenarios if x['mode'] != 'Land_Use']
land_use_scenarios = [x for x in scenarios if x['mode'] == 'Land_Use']

### Calculate skim matrices for new mode scenarios

In the event that the cell below produces a 'Network Travel Times are Zero' error, rebuild the network,
e.g. `ato.build(r'scenario\Transit\box_elder_express.gdb\NetworkDataset\NetworkDataset_ND')`

The cell below should take:

* ~20 minutes per roadway scenario
* ~5 minutes per transit scenario
* ~5 minutes per cycling scenario

In [8]:
# calculate skim matrices for new scenarios
for scenario in modal_scenarios:
    if not arcpy.Exists(os.path.join(scenario['gdb'], "skim_matrix")):
        ato.skim(
            nd = os.path.join(scenario['gdb'], r'NetworkDataset\NetworkDataset_ND'),
            mode = scenario['mode'],
            centroids = r'baseline.gdb\taz_centroids',
            out_table = os.path.join(scenario['gdb'], r"skim_matrix")
        )

Solving skim using Transit network for C:\wfrc\ato\scenario\Transit\redwood_rd_core.gdb\NetworkDataset\NetworkDataset_ND .
Skim matrix written to C:\wfrc\ato\scenario\Transit\redwood_rd_core.gdb\skim_matrix
Skim Matrix Solve Time (mins): 4.84


Just to reiterate, if the loop above fails due to an invalid network (this is caused by a bug in Esri's software, not a problem with your network), you can correct the issue by rebuilding the network.

Insert a new cell and run `ato.build()` on the full network dataset path, e.g. 

```python
ato.build(r'scenario\Transit\ogden_local_bus.gdb\NetworkDataset\NetworkDataset_ND')
```

### Calculate ATO scores

In [15]:
# score mode projects
for scenario in modal_scenarios:
    if not arcpy.Exists(os.path.join(scenario['gdb'], "ato")):
        ato.score(
            skim_matrix = os.path.join(scenario['gdb'], r"skim_matrix"),
            taz_table = r'baseline.gdb\taz_table',
            out_table = os.path.join(scenario['gdb'], r"ato")
        )
for scenario in land_use_scenarios:
    # score land use projects using the baseline skim matrix 
    # since changes in land use do not change travel times
    if scenario['mode'] == 'Land_Use' and not arcpy.Exists(os.path.join(scenario['gdb'], "ato_driving")):
        for mode in ['driving', 'transit', 'cycling']:
            ato.score(
                skim_matrix = os.path.join(baseline_gdb, "skim_" + mode),
                taz_table = os.path.join(gdb, r"taz_table"),
                out_table = os.path.join(gdb, r"ato_" + mode),
                job_per_hh = 1.808755
            )

## Tabulate Scores Across Scenarios

The cells below read in the scores from all scored scenarios and combined into summary tables:

* Transportation Network Changes: `scenario\scenario_scores.csv`
* Land Use Changes: `scenario\land_use_scenario_scores.csv`

Scores for individual projects can be extracted from the `scores_summary` table within each file geodatabase.

### Tabulate Mode Scenario Scores

In [26]:
scenario_scores = pd.DataFrame(columns = 
    ['Name', 'Mode', 'hh_access', 'jobs_access', 'comp_access',
     'pov_accessible_jobs', 'minority_accessible_jobs', 
     'zero_car_accessible_jobs', 'efa_accessible_jobs']
)

equity_taz = pd.read_csv('equity_taz.csv')

for scenario in modal_scenarios:
    
    mode = scenario['mode']
    
    if mode == 'Driving':
        baseline = r'baseline.gdb\ato_driving'
    elif mode == 'Transit':
        baseline = r'baseline.gdb\ato_transit'
    elif mode == 'Cycling':
        baseline = r'baseline.gdb\ato_cycling'
    
    scores_table = os.path.join(scenario['gdb'], 'scores')
    
    if not arcpy.Exists(scores_table):
        ato.diff(
            baseline = baseline,
            scenario = os.path.join(scenario['gdb'], 'ato'),
            out_table = scores_table
        )
    
    df = pd.DataFrame(arcpy.da.TableToNumPyArray(scores_table, '*'))

    df = pd.merge(
        df, 
        equity_taz, 
        on='CO_TAZID', 
        how="left"
    )

    vals = {
        "Name": scenario['name'],
        'Mode': mode,
        "hh_access": df['diff_hh'].sum(),
        "jobs_access": df['diff_jobs'].sum(),
        "comp_access": df['diff_ato'].sum(),
        'pov_accessible_jobs': (df['diff_jobs'] * df['SD_Pov']).sum(),
        'minority_accessible_jobs': (df['diff_jobs'] * df['SD_Minorit']).sum(),
        'zero_car_accessible_jobs': (df['diff_jobs'] * df['SD_ZeroCar']).sum(),
        'efa_accessible_jobs': (df['diff_jobs'] * df[['SD_Pov', 'SD_Minorit', 'SD_ZeroCar']].max(axis=1)).sum()
    }
    
    scenario_scores = scenario_scores.append(vals, ignore_index=True)

Scenario score: 65906.0
Scenario score: 203755.0
Scenario score: 201849.0


In [28]:
scenario_scores

,Name,Mode,hh_access,jobs_access,comp_access,pov_accessible_jobs,minority_accessible_jobs,zero_car_accessible_jobs,efa_accessible_jobs
0,orchard_path,Cycling,45556.0,31371.0,31021.0,4.0,13.0,114.0,127.0
1,penn_ave_buffered_bike_lane,Cycling,84311.0,105388.0,109255.0,24413.0,19676.0,18285.0,41477.0
2,1250_west_new_construction,Driving,31151.0,75811.0,65906.0,43.0,38.0,54.0,82.0
3,highland_drive,Driving,252987.0,176828.0,203755.0,2827.0,1521.0,18399.0,20687.0
4,new_1200_west,Driving,68.0,231.0,-24.0,0.0,0.0,0.0,0.0
5,widen_5600_south,Driving,442572.0,328060.0,392566.0,30901.0,29126.0,38657.0,61369.0
6,box_elder_express,Transit,47922.0,77766.0,65753.0,17708.0,16885.0,22681.0,36075.0
7,ogden_weber_state_brt,Transit,55839.0,118942.0,92481.0,55146.0,21795.0,45259.0,71833.0
8,redwood_rd_core,Transit,124200.0,235469.0,201849.0,96204.0,145160.0,86893.0,180129.0


### Tabulate Land Use Scenario Scores

In [24]:
land_use_scenario_scores = pd.DataFrame(columns = 
    ['Name', 'driving_comp', 'cycling_comp', 'transit_comp', 
     'cycling_to_auto', 'transit_to_auto']
)

for scenario in land_use_scenarios:
    scores = {}

    for mode in ['driving', 'transit', 'cycling']:
        baseline = os.path.join('baseline.gdb', 'ato_' + mode)
        scenario_scores = os.path.join(scenario['gdb'], 'ato_' + mode)

        scores[mode] = ato.diff(baseline, scenario_scores)

    vals = {
        "Name": scenario['name'],
        'driving_comp': scores['driving'],
        'cycling_comp': scores['cycling'],
        'transit_comp': scores['transit'], 
        'cycling_to_auto': round(scores['cycling'] / scores['driving'], 2),
        'transit_to_auto': round(scores['transit'] / scores['driving'], 2)
    }
    
    land_use_scenario_scores = land_use_scenario_scores.append(vals, ignore_index=True)

Scenario score: 884873.0
Scenario score: 45179.0
Scenario score: 166668.0
Scenario score: 400548.0
Scenario score: 22896.0
Scenario score: 83891.0
Scenario score: 275231.0
Scenario score: 46231.0
Scenario score: 63101.0
Scenario score: 844852.0
Scenario score: 88999.0
Scenario score: 143695.0


In [25]:
land_use_scenario_scores

,Name,driving_comp,cycling_comp,transit_comp,cycling_to_auto,transit_to_auto
0,daybreak_commercial_1,884873.0,166668.0,45179.0,0.19,0.05
1,daybreak_commercial_2,400548.0,83891.0,22896.0,0.21,0.06
2,ogden_cbd,275231.0,63101.0,46231.0,0.23,0.17
3,slc_powerplant_redevelopment,844852.0,143695.0,88999.0,0.17,0.11


In [ ]:
land_use_scenario_scores.to_csv(r'scenario\land_use_scenario_scores.csv')
scenario_scores.to_csv(r'scenario\scenario_scores.csv')